In [1]:
import pickle
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.sampling.lhs import LHS
from pymoo.termination.default import DefaultMultiObjectiveTermination
from pymoo.termination import get_termination

from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
import numpy as np

In [2]:
model = pickle.load( open( "bugModel.p", "rb" ) )

In [3]:
class MyProblem(Problem):
    def __init__(self, dim, ub, lb, model):
        super().__init__(
            n_var=dim, n_obj=2, n_constr=0, xl=lb, xu=ub
        )
        self.model = model

    def _evaluate(self, x, out, *args, **kwargs):
        mu, std = self.model._predict(x)
        f1 = mu
        f2 = -std

        # out["F"] = [f1, f2]
        out["F"] = np.column_stack([f1, f2])

In [4]:
dim = 10
lb = np.empty(dim)
lb.fill(-dim)
ub = lb * -1

In [9]:
problem = MyProblem(len(ub), ub, lb, model)



mutation = PolynomialMutation(prob = 1/len(ub), eta=20)
crossover = SBX(prob_var=0.8, eta=20)
algorithm = NSGA2(pop_size=100 * len(ub),
                    crossover=crossover,
                    mutation=mutation,
                    sampling=LHS(),
                eliminate_duplicates=True)

termination = DefaultMultiObjectiveTermination(
    n_max_gen=50,
    n_max_evals=5000 * len(ub))
res = minimize(problem,
        algorithm,
        termination,
        seed=123,
        verbose=True)


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |     1000 |      2 |             - |             -


/Users/aaron/Documents/PhD/PymooBug/scripts/gp.py:35: RuntimeWarning: overflow encountered in multiply
  var = pred.variance.numpy() * (self.y_scaler.std_dev**2)
/Users/aaron/Documents/PhD/PymooBug/scripts/gp.py:35: RuntimeWarning: overflow encountered in multiply
  var = pred.variance.numpy() * (self.y_scaler.std_dev**2)
/Users/aaron/miniconda3/envs/pymooBug/lib/python3.11/site-packages/pymoo/algorithms/moo/nsga2.py:175: RuntimeWarning: invalid value encountered in subtract
  dist = np.row_stack([_F, np.full(n_obj, np.inf)]) - np.row_stack([np.full(n_obj, -np.inf), _F])
/Users/aaron/miniconda3/envs/pymooBug/lib/python3.11/site-packages/pymoo/algorithms/moo/nsga2.py:183: RuntimeWarning: invalid value encountered in divide
  dist_to_last, dist_to_next = dist_to_last[:-1] / norm, dist_to_next[1:] / norm
/Users/aaron/miniconda3/envs/pymooBug/lib/python3.11/site-packages/pymoo/termination/ftol.py:14: RuntimeWarning: invalid value encountered in subtract
  return np.max(np.abs((a - b) / nor

AssertionError: 